In [1]:
%matplotlib inline


# Decoding of a dataset after GLM fit for signal extraction

Full step-by-step example of fitting a GLM to perform a decoding experiment.
We use the data from one subject of the Haxby dataset.

More specifically:

1. Download the Haxby dataset.
2. Extract the information to generate a glm representing the blocks of stimuli.
3. Analyze the decoding performance using a classifier.

To run this example, you must launch IPython via ``ipython
--matplotlib`` in a terminal, or use the Jupyter notebook.
    :depth: 1


## Fetch example Haxby dataset
We download the Haxby dataset
This is a study of visual object category representation



In [1]:
import ast
import gzip
import io
import json
import nilearn
import os
import tarfile
import tempfile
import nibabel as nib
import numpy as np
import pandas as pd
import seaborn as sns
from bids_validator import BIDSValidator

from fetch_difumo import fetch_difumo

from io import BytesIO
from pathlib import Path
from os import listdir as ls
from os.path import basename as bname
from os.path import dirname as dname
from os.path import expanduser as xpu
from os.path import join as pjoin
from pandas import DataFrame as df
from tempfile import TemporaryDirectory as tmpdir
from tempfile import TemporaryFile as tmpfile
from tqdm import tqdm
from typing import Union

import loadutils as lu
import sniffbytes as snif
import scanzip as szip
import shutil

from nilearn import masking
from nilearn.plotting import plot_stat_map, plot_anat, plot_img, plot_epi
from nilearn.image import concat_imgs, mean_img
from nilearn.input_data import NiftiMasker

def lst_intersection(lst1, lst2):
    '''
    Source: https://www.geeksforgeeks.org/python-intersection-two-lists/
    '''
    return [value for value in lst1 if value in set(lst2)]

def read_json(fpath:Union[str, os.PathLike]) -> dict:
    with open(fpath, mode='r', encoding='UTF-8') as jfile:
        jdict = json.load(jfile)
    jfile.close()
    return jdict

cimaq_nov_dir = xpu('~/../../data/cisl/DATA/cimaq_20190901')
cimaq_mar_dir = xpu('~/../../data/cisl/DATA/cimaq_03-19')
events_path = xpu('~/../../data/cisl/DATA/cimaq_corrected_events/events')
behav_path = xpu('~/../../data/cisl/DATA/cimaq_corrected_behavioural/behavioural')
participants_desc = read_json(fpath=pjoin(cimaq_mar_dir, 'participants.json'))
dataset_desc = read_json(fpath=pjoin(cimaq_mar_dir, 'dataset_description.json'))
# post_scan_desc = read_json(fpath=pjoin(cimaq_mar_dir, 'derivatives/CIMAQ_fmri_memory/data/task_files/PostScanBehav_CIMAQ_memory.json'))
# taskfile_headers = read_json(pjoin(cimaq_mar_dir, 'derivatives/CIMAQ_fmri_memory/data/task_files/TaskFile_headers_CIMAQ_memory.json'))
# sorted(ls(pjoin(cimaq_mar_dir, 'derivatives/CIMAQ_fmri_memory/data/features/beta_maps')))
task_results = pd.read_csv(pjoin(cimaq_mar_dir, 'derivatives/CIMAQ_fmri_memory/data/participants/TaskResults/fMRI_behavMemoScores.tsv'),
                           sep='\t')
MemoTaskParticipantFile = pd.read_csv(pjoin(cimaq_mar_dir, 'derivatives/CIMAQ_fmri_memory/data/participants/MemoTaskParticipantFile.tsv'),
                           sep='\t')
# MemoTaskParticipantFile
# task_results.columns
# sorted(ls(pjoin(cimaq_mar_dir, 'derivatives/CIMAQ_fmri_memory/data/participants/')))


/home/fnadeau/myenv/lib/python3.7/site-packages/nilearn/datasets/__init__.py:89: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


In [4]:
class participant_data:
    from nilearn import masking
    from nilearn.plotting import plot_stat_map, plot_anat, plot_img, plot_epi
    from nilearn.image import concat_imgs, mean_img
    def __init__(self):
        cimaq_nov_dir = xpu('~/../../data/cisl/DATA/cimaq_20190901')
        cimaq_mar_dir = xpu('~/../../data/cisl/DATA/cimaq_03-19')
        events_path = xpu('~/../../data/cisl/DATA/cimaq_corrected_events/events')
        behav_path = xpu('~/../../data/cisl/DATA/cimaq_corrected_behavioural/behavioural')
#     def __init__(self, sub_id, mar_scans, nov_scans, mar_infos, nov_infos):
    
        # Load participants infos and indexing file
        participants = pd.read_csv(pjoin(cimaq_mar_dir,
                                         pjoin(cimaq_mar_dir,
                                               'derivatives/CIMAQ_fmri_memory/data/participants/Participants_bids.tsv')),
                                   sep = '\t')
        # Assing each participant to its double identifier
        subjects = df(tuple(('sub-'+str(itm[0]), 'sub-'+str(itm[1])) for itm in
                            tuple(zip(participants.participant_id, participants.pscid))),
                      columns = ['mar_subs', 'nov_subs'])

        # Remove participants who failed quality control
        task_qc = tuple('sub-'+str(itm[0]) for itm in
                        pd.read_csv(pjoin(cimaq_mar_dir, \
                                          'derivatives/CIMAQ_fmri_memory/data/participants/sub_list_TaskQC.tsv'),
                              sep = '\t').values)
        subjects = subjects.iloc[[row[0] for row in subjects.iterrows()
                                        if row[1].mar_subs in task_qc]]

        # Select a random participant
        self.sub_id = subjects.sample(1).values.flatten()
        # Sort march scans and november scans in their respective DataFrames

        mar_scans = lu.loadfiles([itm for itm in
                                    lu.loadimages(pjoin(cimaq_mar_dir, self.sub_id[0]))
                                    if not itm.endswith('.json')])
        nov_scans = lu.loadfiles([itm for itm in
                                    lu.loadimages(pjoin(cimaq_nov_dir, self.sub_id[1]))
                                    if not itm.endswith('.json')])
        self.mar_scans = df(((grp, mar_scans.groupby('parent').get_group(grp).fpaths.values)
                             for grp in mar_scans.groupby('parent').groups)).set_index(0).T

        self.nov_scans = df(((grp, nov_scans.groupby('parent').get_group(grp).fpaths.values)
                             for grp in nov_scans.groupby('parent').groups)).set_index(0).T
        
        mar_infos = lu.loadfiles([itm for itm in
                                    lu.loadimages(pjoin(cimaq_mar_dir, self.sub_id[0]))
                                    if itm.endswith('.json')])
        self.mar_infos = df(((grp, mar_infos.groupby('parent').get_group(grp))
                             for grp in mar_infos.groupby('parent').groups))
        nov_infos = lu.loadfiles([itm for itm in
                                    lu.loadimages(pjoin(cimaq_nov_dir, self.sub_id[0]))
                                    if itm.endswith('.json')])
        self.nov_infos = df(((grp, nov_infos.groupby('parent').get_group(grp))
                             for grp in nov_infos.groupby('parent').groups))
#         self.mar_epi_mask = masking.compute_epi_mask(mean_img(concat_imgs(
#                                 [[img if len(nib.load(img).shape) == 3
#                                   else mean_img(img)][0]
#                                  for img in self.mar_scans.fmap[1]],
#                                 auto_resample=True)))
#         self.mar_anat_mask = mean_img(concat_imgs(self.mar_scans.anat[1],
#                                                     auto_resample = True))
        # Load participant's events (in-scan) and behavioural (post-scan) task files

        self.events = [pd.read_csv(pjoin(events_path, itm), sep = '\t')
                       for itm in lu.loadimages(events_path)
                       if bname(itm).split('_')[1] == self.sub_id[0].split('-')[1]][0]
        self.events['duration'] = [abs(row[1].stim_onset - row[1].fix_onset)
                                   for row in self.events.iterrows()]
        self.events = self.events.rename(columns = {'stim_onset': 'onset'})
        self.events['trial_type'] = self.events['category']
        self.behav = [pd.read_csv(pjoin(behav_path, itm), sep = '\t')
                      for itm in lu.loadimages(behav_path)
                      if bname(itm).split('_')[1] == \
                      self.sub_id[1].split('-')[1]][0]
        correctsources = self.events[['oldnumber', 'correctsource']]
        self.behav['correctsource'] = correctsources.correctsource
        self.behav['correctsource'] = [row[1].correctsource if row[1].oldnumber
                                               in lst_intersection(self.events.oldnumber,
                                                                   self.behav.oldnumber)
                                               else np.nan for row in self.behav.iterrows()]
        self.behav['spatial_acc'] = [row[1].spatial_resp == row[1].correctsource
                                             for row in self.behav.iterrows()]
        self.behav['recognition_acc'] = \
             self.behav['recognition_acc'].replace({0: False, 1: True})
        self.behav.recognition_resp = \
             self.behav.recognition_resp.replace({1: 'old', 2:'new'})
        recognition_accuracy = [row[1].category == row[1].recognition_resp
                                for row in self.behav.iterrows()]
        self.behav['recognition_acc'] = self.behav.recognition_resp.values == \
                                                  self.behav.category.values
        def get_outcomes(behav):
            ''' Compute behavioural (outside scanner) trial outcomes.
                "hit" = successful object and position recognition
                "recog_ok_spatial_wrong" = successful object recognition and
                 wrong position recognition
                "false_alarm" = new object misrecognized as old
                "corr_rejection" = new object recognized as new
                "miss" = old object misrecognized as new'''
            responses = []
            for row in behav.iterrows():
                if row[1].recognition_acc and row[1].spatial_acc:
                    responses.append('hit')
                elif row[1].recognition_acc and not row[1].spatial_acc:
                    responses.append('recog_ok_spatial_wrong')
                elif row[1].category == 'new' and row[1].recognition_resp == 'old':
                    responses.append('false_alarm')
                elif row[1].category == 'new' and row[1].recognition_resp == 'new':
                    responses.append('corr_rejection')
                elif row[1].category == 'old' and row[1].recognition_resp == 'new':
                    responses.append('miss')
            return responses
        outcomes = get_outcomes(self.behav)
        self.behav['outcomes'] = outcomes
        self.confounds = [pd.read_csv(itm, sep='\t') for itm in
                          lu.loadimages(pjoin(cimaq_mar_dir,\
                                              'derivatives/CIMAQ_fmri_memory/data/confounds/resample'))
                          if bname(itm).split('_')[1][3:] == self.sub_id[0].split('-')[1]][0]

        def get_tr_nscans_frametimes(
            fpath:Union[str, os.PathLike]
        ) -> tuple:
            # repetition time is 2.5 seconds
            tr = dict(nib.load(fpath).header)['pixdim'][4]
            # the acquisition comprises 128 310
            n_scans = dict(nib.load(fpath).header)['dim'][4]
            # here are the correspoding frame times
            frame_times = np.arange(n_scans) * tr
            return tr, n_scans, frame_times
        self.tr, self.n_scans, self.frame_times = \
             get_tr_nscans_frametimes(self.mar_scans.func[1][0])
        self.resampled_frame_times=np.arange(0, self.frame_times.max(),
                                             self.frame_times.max()/self.events.shape[0])
        
        def get_epi_mask_fromdata(imgs):
            ''' Compute a nilearn.masking.compute_multi_epi_mask from all available epi data
                Gets nilearn.image.mean_img for 3D images
                If an epi image is 4D, then gets mean_img for each 3D vol outputed by
                nilearn.image.iter_img(<4D_epi_img.nii.gz>), concatenates and auto resample
                all obtained 3D volumes and iterates over each concatenated, resampled and averaged
                3D volume to make the epi_imgs:list parameter to be passed to
                nilearn.image.compute_multi_epi_mask

                Parameters
                ----------
                imgs: list
                List of nifti images paths
                - valid epi images should contain the string "_epi" in their path
            '''
            all_resampled_epi_imgs = lu.flatten(list(nilearn.image.iter_img(nilearn.image.concat_imgs(
                                          lu.flatten(list([mean_img(nilearn.image.load_img(img)) if 
                                                           len(nilearn.image.load_img(img).shape)==3
                                                           else[mean_img(vol) for vol in
                                                                nilearn.image.iter_img(nilearn.image.load_img(img))]]
                                                          for img in imgs
                                                          if '_epi' in img)), auto_resample=True))))
            return nilearn.masking.compute_multi_epi_mask(epi_imgs=all_resampled_epi_imgs)
        # Compute epi masks for march and november scans, respectively
        self.mar_epi_mask = get_epi_mask_fromdata(imgs=self.mar_scans.fmap[1])
        self.nov_epi_mask = get_epi_mask_fromdata(imgs=self.nov_scans.fmap[1])
        
#         self.full_epi_mask=get_epi_mask_fromdata(imgs=list(self.mar_scans.fmap[1])+\
#                                                  list(self.nov_scans.fmap[1]))


In [158]:
subject_data.events

,trialnumber,category,trialcode,oldnumber,correctsource,stim_resp,stim_rt,stim_acc,onset,fix_onset,fix_duration,duration,trial_type,outcomes
0,1,ctl,ctl0,NaN,5,2.0,724,0,4.541,7.541,0.5,3.000,ctl,ctl
1,2,enc,enc00,NaN,8,2.0,630,0,8.041,11.040,0.5,2.999,enc,ctl
2,3,enc,enc000,NaN,9,2.0,922,0,11.540,14.540,5.5,3.000,enc,ctl
3,4,ctl,ctl01,NaN,8,2.0,1386,0,20.039,23.039,0.5,3.000,ctl,ctl
4,5,enc,enc01,old61,6,2.0,858,0,23.539,26.539,1.0,3.000,enc,recog_ok_spatial_wrong
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,116,enc,enc75,old51,6,2.0,672,0,702.976,705.975,10.5,2.999,enc,recog_ok_spatial_wrong
116,117,ctl,ctl39,NaN,9,2.0,622,0,716.474,719.474,5.0,3.000,ctl,ctl
117,118,enc,enc76,old26,5,2.0,784,0,724.474,727.473,1.0,2.999,enc,recog_ok_spatial_wrong
118,119,enc,enc77,old27,5,2.0,656,0,728.473,731.473,18.5,3.000,enc,miss


In [5]:
# By default 2nd subject will be fetched
import numpy as np
import pandas as pd
from nilearn import datasets
import warnings
warnings.filterwarnings('ignore')
# haxby_dataset = datasets.fetch_haxby()
subject_data = participant_data()
# repetition has to be known
TR = subject_data.tr

In [140]:
display(subject_data.mar_epi_mask.shape,
        subject_data.nov_epi_mask.shape,
        nib.load(subject_data.mar_scans.func[1][0]).shape,
        nib.load(subject_data.nov_scans.func[1][0]).shape,)

(80, 80, 41)

(80, 80, 41)

(80, 80, 41, 310)

(80, 80, 41, 310)

In [138]:
list(itm for itm in subject_data.nov_scans.func[1].tolist()+\
     subject_data.mar_scans.func[1].tolist()
     if 'memory' in itm)

['/home/fnadeau/../../data/cisl/DATA/cimaq_20190901/sub-7674650/ses-V03/func/sub-7674650_ses-V03_task-memory_bold.nii.gz',
 '/home/fnadeau/../../data/cisl/DATA/cimaq_20190901/sub-7674650/ses-V10/func/sub-7674650_ses-V10_task-memory_bold.nii.gz',
 '/home/fnadeau/../../data/cisl/DATA/cimaq_03-19/sub-729722/ses-4/func/sub-729722_ses-4_task-memory_bold.nii.gz']

## Load the behavioral data



In [107]:
# haxby_dataset = datasets.fetch_haxby()
haxby_dataset = datasets.fetch_haxby(data_dir=xpu('~/../../data/cisl/DATA/nilearn_data/haxy'),
                                     subjects=list(range(1,6)),
                                     fetch_stimuli=True,
                                     resume=True,
                                     verbose=1)
behavioral_haxby = pd.read_csv(haxby_dataset.session_target[0], sep=' ')
behavioral_haxby.labels.unique(),behavioral_haxby.chunks.unique()

(array(['rest', 'scissors', 'face', 'cat', 'shoe', 'house', 'scrambledpix',
        'bottle', 'chair'], dtype=object),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]))

In [65]:
# Load target information as string and give a numerical identifier to each
behavioral = subject_data.behav
conditions = behavioral['outcomes'].values

# Record these as an array of sessions
sessions = behavioral['oldnumber'].values
unique_sessions = behavioral['oldnumber'].unique()

# fMRI data: a unique file for each session
# func_filename = haxby_dataset.func[0]
func_filename = subject_data.mar_scans.func[1][0]
func_imgs=list(img for img in tqdm(list(nilearn.image.iter_img(
                   subject_data.mar_scans.func[1][0])),
                                      desc='iterating over fmri volumes'))
resampled_func=[nilearn.image.resample_to_img(source_img=img,
                                              target_img=subject_data.mar_epi_mask)
                for img in tqdm(list(nilearn.image.iter_img(subject_data.mar_scans.func[1][0])),
                                desc='resampling fmri volumes to epi mask affine and shape')]
masked_func=nilearn.masking.apply_mask(imgs=resampled_func,
                                          mask_img=subject_data.mar_epi_mask)


iterating over fmri volumes: 100%|███████| 310/310 [00:00<00:00, 1189601.32it/s]
resampling fmri volumes to epi mask affine and shape: 100%|█| 310/310 [00:25<00:


In [141]:
# Prepare fmri image file by resampling the time (4th) dimension to be the same as the number of trials
decomp_func=df(zip(subject_data.frame_times,func_imgs,
       resampled_func,masked_func),columns=['frame_times','func_imgs',
                                            'resampled_func','masked_func'])
test=df(pd.cut(decomp_func['frame_times'],
               subject_data.events.shape[0]))
test[['func_imgs','resampled_func','masked_func']] = \
    decomp_func[['func_imgs','resampled_func','masked_func']].values
results = df(((grp, nilearn.image.mean_img(test.groupby('frame_times').get_group(grp)['resampled_func']),
               test.groupby('frame_times').get_group(grp)['masked_func'],
               nilearn.image.mean_img(test.groupby('frame_times').get_group(grp)['func_imgs']))
              for grp in tqdm(list(test.groupby('frame_times').groups),
                      desc='grouping fmri volumes by events/trials')),
            columns=['intervals','grouped_resampled_func','grouped_masked_func','grouped_func_imgs'])
resampled_frame_times=np.arange(0, subject_data.frame_times.max(),
                                subject_data.frame_times.max()/subject_data.events.shape[0])
subject_data.resampled_frame_times=resampled_frame_times
subject_data.events.outcomes=subject_data.events.outcomes.fillna('ctl')
results['resampled_frame_times']=subject_data.resampled_frame_times
results=results.rename(columns={'func_imgs':'resampled_func_imgs'})
newmatrix=pd.concat([results,subject_data.events],axis=1)

grouping fmri volumes by events/trials: 100%|█| 120/120 [00:55<00:00,  2.17it/s]


In [149]:
newmatrix.columns

Index(['intervals', 'grouped_resampled_func', 'grouped_masked_func',
       'grouped_func_imgs', 'resampled_frame_times', 'trialnumber', 'category',
       'trialcode', 'oldnumber', 'correctsource', 'stim_resp', 'stim_rt',
       'stim_acc', 'onset', 'fix_onset', 'fix_duration', 'duration',
       'trial_type', 'outcomes'],
      dtype='object')

In [157]:
newmatrix.columns
model_events=newmatrix[['oldnumber','resampled_frame_times','trial_type']]
model_events['duration']=model_events['resampled_frame_times'].diff()
model_events=model_events.rename(columns={'resampled_frame_times':'onset'})
model_events[['onset','duration','trial_type']]

,onset,duration,trial_type
0,0.000000,NaN,ctl
1,6.437498,6.437498,enc
2,12.874995,6.437498,enc
3,19.312493,6.437498,ctl
4,25.749990,6.437498,enc
...,...,...,...
115,740.312218,6.437498,enc
116,746.749715,6.437498,ctl
117,753.187213,6.437498,enc
118,759.624710,6.437498,enc


In [156]:
# help(make_first_level_design_matrix)

# smoothing fwhm value criterion
'''
Source: http://jpeelle.net/mri/image_processing/smoothing.html
One common rule of thumb is that, to render your data approximately normal,
you should smooth with a Gaussian filter approximately three times the size of your voxel.
If your voxel size is 3 x 3 x 3 mm, you would smooth with a 9 mm FWHM Gaussian filter.
'''
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm.first_level import make_first_level_design_matrix
first_level_model = FirstLevelModel(t_r=subject_data.tr,
                                    slice_time_ref=0.0,
                                    hrf_model='glover',
                                    drift_model='cosine',
                                    high_pass=0.01,
                                    drift_order=1,
                                    fir_delays=[0],
#                                     min_onset=subject_data.events.iloc[4].onset,
#                                     mask_img=subject_data.mar_epi_mask,
#                                     target_affine=subject_data.mar_epi_mask.affine,
#                                     target_shape=subject_data.mar_epi_mask.shape,
                                    smoothing_fwhm=dict(nib.load(subject_data.mar_scans.func[1][0]).header)['pixdim'][1]*3,
#                                     memory=Memory(location=None),
                                    memory_level=1,
                                    standardize=True,
                                    signal_scaling=0,
                                    noise_model='ar1',
                                    verbose=0, n_jobs=1,
                                    minimize_memory=False,
                                    subject_label=subject_data.sub_id[0])
fmri_design_matrix = \
    make_first_level_design_matrix(
        frame_times=newmatrix.resampled_frame_times,
#         frame_times=np.arange(0, subject_data.frame_times.max(),
#                                           subject_data.frame_times.max()/subject_data.events.shape[0]),
        events=model_events[['onset','duration','trial_type']],
        hrf_model='glover',
        drift_model='cosine',
#         high_pass=0.01,
        drift_order=1)
#         fir_delays=[0])
#         add_regs=behav_design_matrix.iloc[:,:3],
#         add_reg_names=list(behav_design_matrix.columns)[:3],
#         oversampling=50)
nilearn.plotting.plot_design_matrix(fmri_design_matrix)

KeyError: -1

In [133]:
behavioral = pd.read_csv(haxby_dataset.session_target[0], sep=' ')

def get_events(behavioral,
               TR:float=2.5) -> dict:
    conditions = behavioral['labels'].values

    # Record these as an array of sessions
    sessions = behavioral['chunks'].values
    unique_sessions = behavioral['chunks'].unique()

    # fMRI data: a unique file for each session
    func_filename = haxby_dataset.func[0]
    events = {}
    # events will take  the form of a dictionary of Dataframes, one per session
    for session in unique_sessions:
        # get the condition label per session
        conditions_session = conditions[sessions == session]
        # get the number of scans per session, then the corresponding
        # vector of frame times
        n_scans = len(conditions_session)
        frame_times = TR * np.arange(n_scans)
        # each event last the full TR
        duration = TR * np.ones(n_scans)
        # Define the events object
        events_ = pd.DataFrame(
            {'onset': frame_times,
             'trial_type': conditions_session,
             'duration': duration})
        # remove the rest condition and insert into the dictionary
        events[session] = events_[events_.trial_type != 'rest']
    return events,sessions,unique_sessions
events,sessions,unique_sessions=get_events(behavioral)
events

{0:      onset trial_type  duration
 6     15.0   scissors       2.5
 7     17.5   scissors       2.5
 8     20.0   scissors       2.5
 9     22.5   scissors       2.5
 10    25.0   scissors       2.5
 ..     ...        ...       ...
 110  275.0      chair       2.5
 111  277.5      chair       2.5
 112  280.0      chair       2.5
 113  282.5      chair       2.5
 114  285.0      chair       2.5
 
 [72 rows x 3 columns],
 1:      onset    trial_type  duration
 6     15.0          face       2.5
 7     17.5          face       2.5
 8     20.0          face       2.5
 9     22.5          face       2.5
 10    25.0          face       2.5
 ..     ...           ...       ...
 110  275.0  scrambledpix       2.5
 111  277.5  scrambledpix       2.5
 112  280.0  scrambledpix       2.5
 113  282.5  scrambledpix       2.5
 114  285.0  scrambledpix       2.5
 
 [72 rows x 3 columns],
 2:      onset trial_type  duration
 6     15.0        cat       2.5
 7     17.5        cat       2.5
 8     20.0 

In [112]:
# events01=subject_data.events.copy()
# events01['intervals']=list(decomp_func.intervals.unique())
# outcomes_map=dict(zip(events01.intervals,events01.outcomes))
# oldnumber_map=dict(zip(events01.intervals,events01.oldnumber))
# duration_map=dict(zip(events01.intervals,events01.duration))
# onset_map=dict(zip(events01.intervals,events01.onset))
# maps=tuple((col,dict(zip(events01.intervals,events01[col].values)))
#                 for col in list(col for col in
#                                 events01.columns
#                                 if col != 'intervals'))

## Build a proper event structure for each session



In [169]:
# events = {}
# # events will take  the form of a dictionary of Dataframes, one per session
# for session in unique_sessions:
#     # get the condition label per session
#     conditions_session = conditions[sessions == session]
#     # get the number of scans per session, then the corresponding
#     # vector of frame times
#     n_scans = len(conditions_session)
# #     frame_times = TR * np.arange(n_scans)
#     frame_times = subject_data.frame_times
#     # each event last the full TR
#     duration = TR * np.ones(n_scans)
#     # Define the events object
#     events_ = pd.DataFrame(
#         {'onset': frame_times, 'trial_type': conditions_session, 'duration': duration})
#     # remove the rest condition and insert into the dictionary
#     events[session] = events_[events_.trial_type != 'rest']

In [113]:
# from nilearn.image import index_img
# import pandas as pd
# from nilearn import datasets
# from nilearn.image import new_img_like, load_img, get_data

# fmri_filename = subject_data.mar_scans.func[1][0]
# # labels = pd.read_csv(haxby_dataset.session_target[0], sep=" ")
# labels = subject_data.behav[['oldnumber', 'outcomes']]
# y = labels['outcomes']
# session = labels['oldnumber']

# condition_mask = y.isin(y.unique().tolist())

# # Prepare fmri image file by resampling the time (4th) dimension to be the same as the number of trials
# decomp_func = df(img for img in nilearn.image.iter_img(subject_data.mar_scans.func[1][0]))
# decomp_func['frame_times'] = subject_data.frame_times
# test=df(pd.cut(decomp_func['frame_times'], subject_data.events.shape[0]))
# test['imgs'] = decomp_func[0]

In [114]:
# test = test.rename(columns={'frame_times': 'frame_intervals'})
# test['frame_times'] = [val.mid for val in test.frame_intervals.values]
# # test['intervals_duration'] = test.frame_intervals.diff()
# test.frame_times.unique()

In [41]:



###############################################################################################
binned_fmri_image = nilearn.image.concat_imgs(grouped['nifti_imgs'].values,
                                              auto_resample=True)
fmri_img = index_img(binned_fmri_image, condition_mask)
y, session = y[condition_mask], session[condition_mask]

In [170]:
# def get_events(behavioral,TR:float=2.5):
#     conditions = behavioral['outcomes'].values

#     # Record these as an array of sessions
#     sessions = behavioral['oldnumber'].values
#     unique_sessions = behavioral['oldnumber'].unique()

#     # fMRI data: a unique file for each session
#     func_filename = haxby_dataset.func[0]
#     events = {}
#     # events will take  the form of a dictionary of Dataframes, one per session
#     for session in unique_sessions:
#         # get the condition label per session
#         conditions_session = conditions[sessions == session]
#         # get the number of scans per session, then the corresponding
#         # vector of frame times
#         n_scans = len(conditions_session)
#         frame_times = TR * np.arange(n_scans)
#         # each event last the full TR
#         duration = TR * np.ones(n_scans)
#         # Define the events object
#         events_ = pd.DataFrame(
#             {'onset': frame_times,
#              'trial_type': conditions_session,
#              'duration': duration})
#         # remove the rest condition and insert into the dictionary
#         events[session] = events_[events_.trial_type != 'rest']
#     return events,sessions,unique_sessions
# events,sessions,unique_sessions=get_events(decomp_func02)
# events

{'old64':    onset trial_type  duration
 0    0.0       miss       2.5
 1    2.5       miss       2.5
 2    5.0       miss       2.5,
 'old10':    onset trial_type  duration
 0    0.0       miss       2.5
 1    2.5       miss       2.5
 2    5.0       miss       2.5,
 'old26':    onset trial_type  duration
 0    0.0       miss       2.5
 1    2.5       miss       2.5,
 'old77':    onset trial_type  duration
 0    0.0       miss       2.5
 1    2.5       miss       2.5
 2    5.0       miss       2.5,
 'old63':    onset trial_type  duration
 0    0.0        hit       2.5
 1    2.5        hit       2.5
 2    5.0        hit       2.5,
 'old50':    onset              trial_type  duration
 0    0.0  recog_ok_spatial_wrong       2.5
 1    2.5  recog_ok_spatial_wrong       2.5,
 'old39':    onset              trial_type  duration
 0    0.0  recog_ok_spatial_wrong       2.5
 1    2.5  recog_ok_spatial_wrong       2.5
 2    5.0  recog_ok_spatial_wrong       2.5,
 nan: Empty DataFrame
 Columns: [

In [119]:
newmatrix.outcomes=newmatrix.outcomes.fillna('ctl')
df(zip(newmatrix.resampled_frame_times,
    newmatrix.resampled_frame_times.diff(),
    newmatrix.outcomes),columns=['onset','duration','trial_type'])

,onset,duration,trial_type
0,0.000000,NaN,ctl
1,6.437498,6.437498,ctl
2,12.874995,6.437498,ctl
3,19.312493,6.437498,ctl
4,25.749990,6.437498,recog_ok_spatial_wrong
...,...,...,...
115,740.312218,6.437498,recog_ok_spatial_wrong
116,746.749715,6.437498,ctl
117,753.187213,6.437498,recog_ok_spatial_wrong
118,759.624710,6.437498,miss


## Instantiate and run FirstLevelModel

We generate a list of z-maps together with their session and condition index



## Run the glm on data from each session



In [179]:
from nilearn.image import index_img
for session in tqdm(unique_sessions):
    # grab the fmri data for that particular session
    fmri_session = index_img(func_filename, sessions == session)

    # fit the glm
    first_level_model.fit(fmri_session, events=events[session])

    # set up contrasts: one per condition
    conditions = events[session].trial_type.unique()
    for condition_ in conditions:
        z_maps.append(glm.compute_contrast(condition_))
        conditions_label.append(condition_)
        session_label.append(session)

  9%|███▉                                        | 7/79 [00:17<02:58,  2.48s/it]


IndexError: index -1 is out of bounds for axis 0 with size 0

## Generating a report
Since we have already computed the FirstLevelModel
and have the contrast, we can quickly create a summary report.



In [175]:
from nilearn.image import mean_img
from nilearn.reporting import make_glm_report
mean_img_ = mean_img(func_filename)
report = make_glm_report(glm,
                         contrasts=conditions,
                         bg_img=mean_img_,
                         )

report  # This report can be viewed in a notebook

ValueError: unsupported format character 'n' (0x6e) at index 1

In a jupyter notebook, the report will be automatically inserted, as above.
We have several other ways to access the report:



In [ ]:
# report.save_as_html('report.html')
# report.open_in_browser()

## Build the decoding pipeline
To define the decoding pipeline we use Decoder object, we choose :

    * a prediction model, here a Support Vector Classifier, with a linear
      kernel

    * the mask to use, here a ventral temporal ROI in the visual cortex

    * although it usually helps to decode better, z-maps time series don't
      need to be rescaled to a 0 mean, variance of 1 so we use
      standardize=False.

    * we use univariate feature selection to reduce the dimension of the
      problem keeping only 5% of voxels which are most informative.

    * a cross-validation scheme, here we use LeaveOneGroupOut
      cross-validation on the sessions which corresponds to a
      leave-one-session-out

We fit directly this pipeline on the Niimgs outputs of the GLM, with
corresponding conditions labels and session labels (for the cross validation).



In [ ]:
from nilearn.decoding import Decoder
from sklearn.model_selection import LeaveOneGroupOut
decoder = Decoder(estimator='svc', mask=haxby_dataset.mask, standardize=False,
                  screening_percentile=5, cv=LeaveOneGroupOut())
decoder.fit(z_maps, conditions_label, groups=session_label)

# Return the corresponding mean prediction accuracy compared to chance

classification_accuracy = np.mean(list(decoder.cv_scores_.values()))
chance_level = 1. / len(np.unique(conditions))
print('Classification accuracy: {:.4f} / Chance level: {}'.format(
    classification_accuracy, chance_level))